In [1]:
using LinearAlgebra
import Base: \
# function LinearAlgebra.Bidiagonal(dv::AbstractVector{T}, ev::AbstractVector{S}, uplo::Symbol) where {T,S}
#     TS = promote_type(T,S)
#     Bidiagonal(convert(AbstractVector{TS}, dv),
#                convert(AbstractVector{TS}, ev),
#                 uplo)
# end


# ## The base method narrows the type too much. We'll have to ensure that it's as least as wide as the input
# function  \(adjA::Adjoint{<:Any,<:Union{UnitUpperTriangular,UnitLowerTriangular}}, B::AbstractVector)
#     A = adjA.parent
#     TAB = promote_type(eltype(A), eltype(B), typeof(zero(eltype(A))*zero(eltype(B)) + zero(eltype(A))*zero(eltype(B))))
#     BB = similar(B, TAB, size(B))
#     copyto!(BB, B)
#     ldiv!(adjoint(convert(AbstractArray{TAB}, A)), BB)
# end

In [2]:
h(x) = exp(-x)
h′(x) = -exp(-x)
h′(x,y) = -y
𝓁(x,y) = sum(abs2,x-y)/2
𝓁′(x,y) = x-y
init(sizes...) = 0.01randn(sizes...)

init (generic function with 1 method)

In [3]:
𝜀 = 0.0001
n = [5,4,3,1]
N = length(n)-1
B = 7

7

### Scalar Neural Network

Params: [ $ \ $[$w_i$, $b_i$] $ \ $ for $i=1,...,N$] <br>
Input: $x_1$ <br>
Intermediate: $x_{i+1}=h(w_ix_i+b_i)$ for $i=1,...,N$ <br>
Intermediate: $\delta_i = h^{\prime}(w_ix_i+b_i))$ for $i=1,...,N$<br>
Output: X = [$x_1,\ldots,x_{N+1}$]<br>
Output: δ =  [$\delta_1,\ldots,\delta_N$] <br>
y: data (used in loss function 𝓁(x,y) )

In [4]:
function neural_net(params, input; h=h, h′=h′, N=length(params))
    Δ = [];
    X = [input];
    for i=1:N
        x = sum(params[i] .* [X[i],1])
        push!(X,h(x))
        push!(Δ, h′.(x,X[i+1]))
    end
    return X,Δ
end


neural_net (generic function with 1 method)

In [5]:
params =[[init(),init()] for i=1:N] # W and B
x,y = init(),init() # input and output

(-0.015192320630271504, 0.029450987349552742)

In [6]:
X,δ = neural_net(params,x)
L   = Bidiagonal(zeros(N),[δ[i] * params[i][1] for i=2:N],:L)
D   = Diagonal(δ.*[[X[i],1]' for i=1:N])
g   = [zeros(N-1);𝓁′(X[N+1],y)]
∇J  = D'*((I-L')\g)

3-element Vector{Vector{Float64}}:
 [9.485452549408101e-7, -6.243583702747712e-5]
 [-0.002884495624013203, -0.0028846090763231945]
 [-0.9825250277161762, -0.9610226017210001]

In [7]:
# ∇Jfd is gradient calculated with finite differences method
∇Jfd = ∇J * 0
ϵ    = ∇J * 0
for i=1:N, j=1:2       
    ϵ[i][j] = 𝜀
    ∇Jfd[i][j]=(𝓁(neural_net(params.+ϵ,x)[1][N+1],y)-𝓁(neural_net(params.-ϵ,x)[1][N+1],y))/2𝜀
    ϵ[i][j] = .0
end
∇Jfd

3-element Vector{Vector{Float64}}:
 [9.485451313295812e-7, -6.243583733356317e-5]
 [-0.002884495629018602, -0.0028846090813217096]
 [-0.9825250347195169, -0.9610226082745754]

In [8]:
import Base: +,-,*,/,∘

struct LinearMatrixOp # Is parametric type necessary? It causes un-readable error messages and some other issues.
    f
    fadj
end
LinearMatrixOp(f::Function) = LinearMatrixOp(f,f)

LeftMul(A::AbstractMatrix) = LinearMatrixOp(X->A*X, X->A'*X)


RightMul(A::Union{AbstractMatrix,AbstractVector}) = LinearMatrixOp(X->X*A, X->X*A')
HadMul(A::Union{AbstractMatrix,AbstractVector}) = LinearMatrixOp(X->X.*A)
ZeroMul() = LinearMatrixOp(X->Zero())
IdentMul() = LinearMatrixOp(X->X) #not neccessary, can be commented

Base.zero(::Type{LinearMatrixOp}) = ZeroMul() 
Base.one(::Type{LinearMatrixOp}) = IdentMul()
Base.adjoint(A::LinearMatrixOp) = LinearMatrixOp(A.fadj,A.f)
Base.copy(A::LinearMatrixOp) =  LinearMatrixOp(A.f,A.fadj)

*(A::LinearMatrixOp,X::Union{AbstractArray,Number}) = A.f(X)
-(A::LinearMatrixOp) = LinearMatrixOp(X->-A.f(X), X->-A.fadj(X))
∘(A::LinearMatrixOp, B::LinearMatrixOp) = LinearMatrixOp(A.f ∘ B.f, B.fadj ∘ A.fadj)

# A zero
struct Zero end
Base.zero(::Type{Any}) = Zero()
+(::Zero, ::Zero) = Zero()
-(::Zero, A) = -A
+(::Zero, A) = A
*(::Zero, ::Zero) = Zero()
*(X, ::Zero) = Zero()

* (generic function with 372 methods)

## Bernie wants a vector example

In [9]:
n = [5,4,3,1]
N = length(n)-1
params =[[.1randn(n[i+1],n[i]),.1randn(n[i+1])] for i=1:N]
X₁, y = randn(n[1]), randn(1);

In [10]:
function neural_net(params,input; h=h,h′= h′)
    X     = [input]
    Δ     = []
    for i=1:length(params)
        t = params[i][1]*X[i] .+ params[i][2]         
        push!(X,h.(t))
        push!(Δ,h′.(t))
    end 
    X,Δ
end
array(x)= fill(x,1,1)

array (generic function with 1 method)

In [11]:
X,Δ = neural_net(params,X₁)
W = [params[i][1] for i=1:N]
D = Diagonal([[HadMul(Δ[i]) ∘ RightMul(X[i]) HadMul(Δ[i])] for i=1:N])
I_minus_L = Bidiagonal([I for i in 1:N], -[HadMul(Δ[i]) ∘ LeftMul(W[i]) for i=2:N] , :L)
g = [ [Zero() for i=1:N-1]; [𝓁′(X[N+1],y)] ]
∇J = D'*array.(I_minus_L'\g)

3-element Vector{Matrix{Any}}:
 [[0.010343548314343382 -0.03349325028697286 … 0.014558585395377516 0.016513138118211413; 0.0033082815826314515 -0.010712484700555283 … 0.004656419486715778 0.005281563835514039; 0.022784517975740128 -0.07377812139875331 … 0.0320692996795253 0.036374741129069854; -0.0007857569142793368 0.0025443447639900123 … -0.0011059559823084024 -0.001254435330943584]; [-0.02744930330075746, -0.008779388059708145, -0.06046466120435512, 0.002085210916529661]]
 [[0.13484921906943864 0.11320772738803518 0.1626659890366508 0.08676544692535836; 0.013040128012007256 0.01094736230046856 0.01573005268310395 0.00839035288994206; -0.10686544468774763 -0.08971497360457091 -0.12890970651467146 -0.06875996860193152]; [0.12054682604831742, 0.011657064489945464, -0.09553106988863208]]
 [[-0.5929107928462909 -0.5084669931169483 -1.28955008112464]; [-0.6816245032662562]]

In [12]:
# ∇Jfd gradient finite differences
∇Jfd = params*0
ϵ=params*0
for i=1:length(params), wb=1:2
    for j=1:length(ϵ[i][wb])
        ϵ[i][wb][j] = 𝜀
        ∇Jfd[i][wb][j] =(𝓁(neural_net(params+ϵ,X₁)[1][N+1],y)-𝓁(neural_net(params-ϵ,X₁)[1][N+1],y))/2𝜀
        ϵ[i][wb][j] = .0
     end
end
∇Jfd

3-element Vector{Vector{Array{Float64,N} where N}}:
 [[0.010343548318969553 -0.0334932504383878 … 0.014558585407220548 0.016513138136331484; 0.00330828158312535 -0.010712484722619209 … 0.004656419488324737 0.005281563838221981; 0.022784517987040065 -0.07377812177272736 … 0.032069299709946986 0.036374741174127756; -0.0007857569145341969 0.0025443447712603096 … -0.0011059559831827492 -0.0012544353317212042], [-0.027449303383886292, -0.008779388071811223, -0.060464661410242204, 0.0020852109208768788]]
 [[0.13484921907502834 0.11320772739159546 0.16266598904662866 0.086765446926651; 0.013040128036267973 0.010947362314717157 0.015730052725648758 0.008390352896348041; -0.10686544512639173 -0.08971497386406346 -0.12890970728457463 -0.06875996871852363], [0.12054682605205924, 0.011657064507286652, -0.09553107020221496]]
 [[-0.5929107960633839 -0.5084669951460619 -1.2895501142246024], [-0.6816245081542327]]

Params: [ $ \ $[$W_i$, $b_i$] $ \ $ for $i=1,...,N$].<br>
$W_i$ is $n_{i+1} \times n_i$. <br>
$b_i$ is $n_{i+1}$  <br> <br>
Input: $X_1$ is $n_1 \times B$. ($B$ = Batch size)  <br>
(In the below the dot in matrix plus vector means broadcast to all columns) <br>
Intermediate: $X_{i+1}=h(W_iX_i.+b_i)$ for $i=1,...,N$ <br>
Intermediate: $\delta_i = h^{\prime}(W_ix_i.+b_i))$ for $i=1,...,N$<br>
Output: X = [$X_1,\ldots,X_{N+1}$] (Each $X_i$ is $n_i \times B$) <br>
Output: Δ =  [$\Delta_1,\ldots,\Delta_N$]  (Each $\Delta_i$ is $n_i \times B$) <br>
    y: 1xB (used in loss function 𝓁($X_1$,y) )

In [13]:
# params: `W_i` and `b_i`s: x_{i+1} <- Wi * x_i .+ b_i

n = [5,4,3,1] 
N = length(n)-1
B = 1

params =[[.1randn(n[i+1],n[i]),.1randn(n[i+1])] for i=1:N]
X₁, y = randn(n[1],B), randn(1,B);

$\begin{pmatrix}
        dX_2 \\
        dX_3 \\
        \vdots \\
        dX_N \\
        dX_{N+1}
    \end{pmatrix} = diag
        \begin{pmatrix}
        [H(\Delta_1)  \circ R(X_1) \,\, H(\Delta_1)] \\
        [H(\Delta_1)  \circ R(X_2) \,\, H(\Delta_2)] \\
         \vdots \\
        [H(\Delta_{N-1})  \circ R(X_{N-1}) \,\, H(\Delta_{N-1})] \\ 
         [H(\Delta_{N})  \circ R(X_{N}) \,\, H(\Delta_{N})]
    \end{pmatrix}
    \begin{pmatrix}
        [dW_1; dB_1]\\
        [dW_2; dB_2]\\
        \vdots \\
        [dW_{N-1};dB_{N-1}] \\ 
        [dW_N; dB_N]
    \end{pmatrix}
    $ <br>
    $
    +  \begin{pmatrix}
        0 & \ldots & 0 & 0 & 0 \\
        H(\Delta_2) \circ L(W_2) & \ldots & 0 & 0 & 0 \\
        & \ddots \\
        && H(\Delta_{N-1}) \circ L(W_{N-1}) & 0 & 0\\
        &&& H(\Delta_{N}) \circ L(W_{N})& 0 \\
    \end{pmatrix}
     \begin{pmatrix}
        dX_2 \\
        dX_3 \\
        \vdots \\
        dX_N \\
        dX_{N+1}
    \end{pmatrix}
    $

In [14]:
X,Δ = neural_net(params,X₁)
W = [params[i][1] for i=1:N]
D = Diagonal([[HadMul(Δ[i]) ∘ RightMul(X[i]) HadMul(Δ[i])] for i=1:N])
I_minus_L = Bidiagonal([I for i in 1:N], -[HadMul(Δ[i]) ∘ LeftMul(W[i]) for i=2:N] , :L)
g = [ [Zero() for i=1:N-1]; [𝓁′(X[N+1],y)] ]
∇J = D'*array.(I_minus_L'\g);

In [15]:
X[1]

5×1 Matrix{Float64}:
 -0.7609461743103296
  0.06376474224006651
  0.24165987100463138
  0.7322391447630908
 -0.07482665336968171

In [16]:
# ∇Jfd gradient finite differences
∇Jfd = params*0
ϵ=params*0
for i=1:length(params), wb=1:2
    for j=1:length(ϵ[i][wb])
        ϵ[i][wb][j] = 𝜀
        ∇Jfd[i][wb][j] =(𝓁(neural_net(params+ϵ,X₁)[1][N+1],y)-𝓁(neural_net(params-ϵ,X₁)[1][N+1],y))/2𝜀
        ϵ[i][wb][j] = .0
     end
end
∇Jfd;

In [17]:
∇J[1,1][1]

4×5 Matrix{Float64}:
 -0.0110386     0.000924994   0.0035056     0.0106221    -0.00108546
 -0.0063253     0.000530039   0.00200878    0.00608667   -0.00062199
  0.000103981  -8.71323e-6   -3.3022e-5    -0.000100058   1.02248e-5
 -0.00160644    0.000134614   0.000510169   0.00154583   -0.000157967

In [18]:
∇Jfd[1,1][1]

4×5 Matrix{Float64}:
 -0.0110386     0.000924994   0.0035056     0.0106221    -0.00108546
 -0.0063253     0.000530039   0.00200878    0.00608667   -0.00062199
  0.000103981  -8.71323e-6   -3.3022e-5    -0.000100058   1.02248e-5
 -0.00160644    0.000134614   0.000510169   0.00154583   -0.000157967

### Matrix Neural Network

In [19]:
n = [5,4,3,1]
N = length(n)-1
𝜀 = 0.0001
B = 7

7

In [20]:
function neural_net(params,input;h=h,h′=h′)
    X     = [input]
    δ     = []
    for i=1:length(params)
        x = params[i][1]*X[i] .+ params[i][2]         
        push!(X,h.(x))
        push!(δ,h′.(x,X[i+1]))
    end 
    X,δ
end
array(x)= fill(x,1,1)

array (generic function with 1 method)

params: `W_i` and `b_i`s: x_{i+1} <- Wi*x_i .+ b_i

In [21]:
params =[[init(n[i+1],n[i]),init(n[i+1])] for i=1:N]
x, y = init(n[1],B), init(1,B);

In [22]:
X,δ = neural_net(params,x)
D = Diagonal([[HadMul(δ[i]) ∘ RightMul(X[i]) HadMul(δ[i])] for i=1:N])
ImL = Bidiagonal([I for i in 1:N], -[HadMul(δ[i]) ∘ LeftMul(params[i][1]) for i=2:N] , :L)
g = [ [Zero() for i=1:N-1]; [𝓁′(X[N+1],y)] ] 
∇J = D'*array.(ImL'\g)

3-element Vector{Matrix{Any}}:
 [[1.2500176587353581e-5 6.590525686816944e-6 … -6.113225135061867e-6 -7.801383448250018e-6; -1.497817760603035e-6 -7.893995308292463e-7 … 7.326570720015704e-7 9.347948871601612e-7; -1.3118352365875809e-5 -6.919726649867388e-6 … 6.4180757058971916e-6 8.184254421786065e-6; 2.6964481150250814e-6 1.4220392094734789e-6 … -1.3193220048995069e-6 -1.6840438370848367e-6]; [0.0002816118885992652 0.0002831657750439727 … 0.00028085789964974454 0.000278715943704693; -3.3732806094589646e-5 -3.3910179095024034e-5 … -3.36451232528617e-5 -3.338860585847111e-5; -0.00029553494209633786 -0.00029715254303487533 … -0.00029461189284070324 -0.00029248365031532477; 6.076446415123484e-5 6.109104356353871e-5 … 6.056784978155905e-5 6.0138290040488675e-5]]
 [[-0.11914481927960491 -0.11939400217154643 -0.12142031083673799 -0.11862043357437975; -0.04098923255791988 -0.04107495861086029 -0.04177206685944679 -0.04080882881724512; 0.012261922481493994 0.012287567409821697 0.0124961072399

In [23]:
# ∇Jfd is gradient calculated with finite differences method
∇Jfd = params*0
ϵ=params*0
for i=1:length(params), wb=1:2
    for j=1:length(ϵ[i][wb])
        ϵ[i][wb][j] = 𝜀
        ∇Jfd[i][wb][j] =(𝓁(neural_net(params+ϵ,x)[1][N+1],y)-𝓁(neural_net(params-ϵ,x)[1][N+1],y))/2𝜀
        ϵ[i][wb][j] = .0
     end
end
∇Jfd;

In [24]:
∇Jfd[1][1]

4×5 Matrix{Float64}:
  1.25002e-5   6.59053e-6   7.57094e-7  -6.11322e-6  -7.80139e-6
 -1.49782e-6  -7.89402e-7  -9.05409e-8   7.32658e-7   9.34797e-7
 -1.31184e-5  -6.91972e-6  -7.98037e-7   6.41807e-6   8.18425e-6
  2.69645e-6   1.42204e-6   1.63012e-7  -1.31932e-6  -1.68404e-6

In [25]:
∇J[1][1]

4×5 Matrix{Float64}:
  1.25002e-5   6.59053e-6   7.57093e-7  -6.11323e-6  -7.80138e-6
 -1.49782e-6  -7.894e-7    -9.05418e-8   7.32657e-7   9.34795e-7
 -1.31184e-5  -6.91973e-6  -7.98037e-7   6.41808e-6   8.18425e-6
  2.69645e-6   1.42204e-6   1.63011e-7  -1.31932e-6  -1.68404e-6

### A Showcase: Densely Connected Matrix Network

In [26]:
function neural_net(params,input;h=h, h′= h′)
    X     = [input]
    δ     = []
    for i in 1:length(params)
       x = broadcast(+,(params[i] .* [X..., I])...)
       push!(X,h.(x))
       push!(δ,h′.(x,X[i+1]))
    end 
    X,δ
end;
array(x) = fill(x,1,1);

In [27]:
params = [[j==i+1 ?  init(n[i+1],1) : init(n[i+1],n[j])  for j=1:i+1] for i=1:N]
x,y = init(n[1],B), init(1,B);

In [28]:
X,δ = neural_net(params,x)
D = Diagonal([[[(HadMul(δ[i]) ∘ RightMul(X[j]))' for j=1:i]' HadMul(δ[i])] for i=1:N])
ImL = UnitLowerTriangular(Matrix{Any}(undef,N,N))
for i=2:N, j=1:i-1
    ImL[i,j] = -HadMul(δ[i]) ∘ LeftMul(params[i][j+1]) 
end
g =[ [Zero() for i=1:N-1]; [𝓁′(X[N+1],y)] ] 
∇J = D'*array.(ImL'\g)

3-element Vector{Matrix{Any}}:
 [[8.108631009110678e-6 8.080304654661267e-6 … -1.1903004166362233e-5 -4.509977123148807e-6; 0.0001569435563928751 0.0001563036807495304 … -0.00023036396471757455 -8.727210947246424e-5; -4.385086358119289e-5 -4.3667337959258324e-5 … 6.434376408235784e-5 2.4376731382031054e-5; -0.00031353109166912176 -0.00031228829982786574 … 0.000460202724885639 0.00017439672331565542]; [0.00037949609136086673 0.0003885390442528399 … 0.0003809831692149818 0.00038318582221509085; 0.007343805643799245 0.007518728923223524 … 0.007375438607441067 0.007416708692183254; -0.0020518412290114843 -0.002100025709284073 … -0.002059618845014968 -0.002072053456902055; -0.01467356577043743 -0.015019076984295136 … -0.014734312002491796 -0.014818622930994889]]
 [[2.3220833299198935e-5 2.3113300575941924e-5 … -3.4068091493727626e-5 -1.2903865722413128e-5; -5.520694331066404e-5 -5.496383785617488e-5 … 8.099538606053649e-5 3.067865094392669e-5; 0.00014739321149422594 0.00014686552504624686 …

In [29]:
# ∇Jfd is gradient calculated with finite differences method
∇Jfd = params*0
ϵ=params*0
for i=1:length(ϵ), j=1:length(ϵ[i]), k=1:length(ϵ[i][j])
        ϵ[i][j][k] = 𝜀
        ∇Jfd[i][j][k] =(𝓁(neural_net(params+ϵ,x)[1][N+1],y)-𝓁(neural_net(params-ϵ,x)[1][N+1],y))/2𝜀
        ϵ[i][j][k] = .0
end

In [30]:
∇Jfd[1][1]

4×5 Matrix{Float64}:
  8.10863e-6    8.08031e-6   -2.16377e-6  -1.1903e-5    -4.50998e-6
  0.000156944   0.000156304  -4.19521e-5  -0.000230364  -8.72721e-5
 -4.38509e-5   -4.36673e-5    1.17407e-5   6.43438e-5    2.43767e-5
 -0.000313531  -0.000312288   8.37816e-5   0.000460203   0.000174397

In [ ]:
∇J[1][1]

## Neural Network with Orthogonal Parameters

In [ ]:
n = [2,2,2,1]
N = length(n)-1
B = 7
h(x) = exp(-x)
h′(x,y) = -y
𝓁(x,y) = sum(abs2,x-y)/2
𝓁′(x,y) = x-y
init(sizes...) = 0.01randn(sizes...)

In [ ]:
struct OrthogonalTransform{T} <: AbstractMatrix{T}
    θ::T
end
Base.Matrix{T}(R::OrthogonalTransform{T}) where T =   [cos(R.θ) -sin(R.θ); sin(R.θ) cos(R.θ)] 
*(R::OrthogonalTransform, x::AbstractArray{T,2} where T) = [cos(R.θ) -sin(R.θ); sin(R.θ) cos(R.θ)]  * x
Base.adjoint(R::OrthogonalTransform) = OrthogonalTransform(-R.θ)
Base.size(R::OrthogonalTransform) = (2,2)
Base.getindex(R::OrthogonalTransform, inds...) = [cos(R.θ) -sin(R.θ); sin(R.θ) cos(R.θ)][inds...]
-(R::OrthogonalTransform{T}) where T = OrthogonalTransform{T}(-R.θ)
Ort(Q::OrthogonalTransform) = LinearMatrixOp(X->Q*X, X->(X-Q*X'*Q)/2) # onl definition needed so far. So I left f as identity

In [ ]:
function neural_net(params,input;h=h,h′= h′)
    X     = [input]
    δ     = []
    for i=1:length(params)
        x = params[i][1]*X[i] .+ params[i][2]         
        push!(X,h.(x))
        push!(δ,h′.(x,X[i+1]))
    end 
    X,δ
end
array(x)= fill(x,1,1)

In [ ]:
OrthogonalTransform(π/rand([1,2,3,4,6])) 

In [ ]:
# params: `W_i` and `b_i`s: x_{i+1} <- Wi*x_i .+ b_i
params =[[(i!=N ? OrthogonalTransform(π/rand([1,2,3,4,6])) : init(n[i+1],n[i])), init(n[i+1])] for i=1:N]
x, y = init(n[1],B), init(1,B);

In [ ]:
X,δ = neural_net(params,x)
D = Diagonal([[HadMul(δ[i]) ∘ RightMul(X[i]) ∘  (i!=N ? Ort(params[i][1]) : IdentMul())     HadMul(δ[i])] for i=1:N])
ImL = Bidiagonal([I for i in 1:N], -[HadMul(δ[i]) ∘ LeftMul(params[i][1]) for i=2:N] , :L)
g = push!(Any[Zero() for i=1:N-1],𝓁′(X[N+1],y))
∇J = D'*array.(ImL'\g)

In [ ]:
# Test
diff = Matrix(OrthogonalTransform(params[1][1].θ + 𝜀)) - Matrix(params[1][1])
@show Δ = sum(diff.* ∇J[1][1])
orig    = 𝓁(neural_net(params,x)[1][N+1],y)
params[1][1] = OrthogonalTransform(params[1][1].θ + 𝜀)
updated = 𝓁(neural_net(params,x)[1][N+1],y)
params[1][1] = OrthogonalTransform(params[1][1].θ - 𝜀)
@show Δ = updated-orig;

In [ ]:
struct HouseHolderTransform{T} <: AbstractMatrix{T}
    mat::Matrix{T}
end
HouseHolderTransform(v::Vector{T}) where T = HouseHolderTransform(I-2*v*v')
Base.Matrix{T}(R::HouseHolderTransform{T}) where T =   R.mat
*(R::HouseHolderTransform, x::AbstractArray{T,2} where T) =  R.mat* x
Base.adjoint(R::HouseHolderTransform) = HouseHolderTransform(permutedims(R.mat))
Base.size(R::HouseHolderTransform) = size(R.mat)
Base.getindex(R::HouseHolderTransform, inds...) = R.mat[inds...]
-(R::HouseHolderTransform) = -R.mat
Ort(Q::AbstractMatrix) = LinearMatrixOp(X->Q*X, X->(X-Q*X'*Q)/2)

In [ ]:
n = [4,4,4,1]
N = length(n)-1
B = 7

In [ ]:
# params: `W_i` and `b_i`s: x_{i+1} <- Wi*x_i .+ b_i
vs = [init(n[i]) for i=1:N-1] # householder parameters
vs = map(v-> (v ./ norm(v)), vs)
params =[[(i!=N ? HouseHolderTransform(vs[i]) : init(n[i+1],n[i])), init(n[i+1])] for i=1:N]
x, y = init(n[1],B), init(1,B);

In [ ]:
X,δ = neural_net(params,x)
D = Diagonal([[HadMul(δ[i]) ∘ RightMul(X[i]) ∘  (i!=N ? Ort(params[i][1]) : IdentMul())     HadMul(δ[i])] for i=1:N])
ImL = Bidiagonal([I for i in 1:N], -[HadMul(δ[i]) ∘ LeftMul(params[i][1]) for i=2:N] , :L)
g = push!(Any[Zero() for i=1:N-1],𝓁′(X[N+1],y))
∇J = D'*array.(ImL'\g)

In [ ]:
∇J[1][1]

In [ ]:
# Test
vs1_changed = vs[1] .+ 𝜀
vs1_changed = vs1_changed ./ norm(vs1_changed)
#vs1_changed[1] = vs[1][1] + 𝜀
diff = Matrix(HouseHolderTransform(vs1_changed)) - Matrix(params[1][1])
@show Δ = sum(diff.* ∇J[1][1])
orig    = 𝓁(neural_net(params,x)[1][N+1],y)
params[1][1] = HouseHolderTransform(vs1_changed)
updated = 𝓁(neural_net(params,x)[1][N+1],y)
params[1][1] = HouseHolderTransform(vs[1])
@show Δ = updated-orig;

In [ ]:
vs1_changed 

## MNIST MLP Example (didn't test on final version)

In [ ]:
# Data
using Knet
import Knet: Data
include(Knet.dir("data","mnist.jl"))
dtrn,dtst = mnistdata(xsize=(784,:)); # dtrn and dtst = [ (x1,y1), (x2,y2), ... ] where xi,yi are

#Layers
n = [784,128,64,10]
N = length(n)-1
init(sizes...) = 0.1randn(sizes...)

#Nonlinearity
h(x)    = x>0 ? x : zero(x) # relu
h′(x,y) = y>0 ? one(x) : zero(x) # derivative of relu

#Loss
𝓁(x,a) = nll(x,a;average=true) # negative log likelihood loss, x is dxb matrix, 
                               # a is d-length integer array keeps the correct answers 
function 𝓁′(x,a)  # Note!: this will be simplified if we can figure out how to integrate derivative of getindex in to our formulatin
    indices = Knet.findindices(x,a,dims=1)
    yz = zero(x)
    yz[indices] .= 1
    return (softmax(x,dims=1) .- yz)./length(a)
end

#Forward Function
function neural_net(params,input;h=h,h′= h′)
    X     = [input]; δ     = []
    for i=1:length(params)-1
        x = params[i][1]*X[end] .+ params[i][2]         
        push!(X,h.(x)); push!(δ,h′.(x,X[end]))
    end 
    x = params[end][1]*X[end] .+ params[end][2]    
    push!(X,x); push!(δ,one.(x))
    X,δ
end

In [ ]:
params =[[init(n[i+1],n[i]),zeros(n[i+1])] for i=1:N] # model parameters
α = 0.5 # learning rate 
epochs=3# number of epochs to train model 
@time for i=1:epochs # 1 epoch takes ~ 65 seconds  in my macbook
    for (x,y) in dtrn
        X,δ = neural_net(params,x;h=h, h′= h′)
        D = Diagonal([[HadMul(δ[i]) ∘ RightMul(X[i]) HadMul(δ[i])] for i=1:N])
        ImL = Bidiagonal([I for i in 1:N], -[HadMul(δ[i]) ∘ LeftMul(params[i][1]) for i=2:N] , :L)
        g = push!(Any[Zero() for i=1:N-1],𝓁′(X[N+1],y))
        ∇J = D'*array.(ImL'\g);
        for i =1:length(params)
            params[i][1] = params[i][1] - α*∇J[i][1]
            params[i][2] = params[i][2] - α*sum(∇J[i][2],dims=2)
        end
    end
end

In [ ]:
zeroone=total=0
for (x,y) in dtst
    yn        = neural_net(params,x;h=h, h′= h′)[1][end]
    answers   = vec(getindex.(argmax(yn,dims=1),1))
    global zeroone += sum(y .== answers)
    global total   += length(answers)
end
accuracy = 100zeroone/total